In [286]:
import requests
import json
import os
import pandas as pd
from pprint import pprint
from dotenv import load_dotenv

load_dotenv()

True

In [287]:
base_url = "https://jira.eficode.com"
access_token = os.getenv('JIRA_API_TOKEN')


In [288]:
url = f'{base_url}/rest/tempo-planning/1/allocation'
header = {'Authorization': f'Bearer {access_token}', 'Content-Type': 'application/json'}
res = requests.get(url=url, headers=header)

print(res.status_code)
data = res.json()

200


In [289]:
df = pd.DataFrame(columns=['plan_id', 'employee', 'start', 'end', 'hours_day', 'employee_day'])

for plan in data:
    row = {'plan_id': plan['id'], 'employee': plan['assignee']['key'], 'start': plan['start'], 'end': plan['end'], 'hours_day': int(plan['secondsPerDay'])/(60*60)}
    df.loc[len(df)] = row

df['start'] = pd.to_datetime(df['start'], format='%Y/%m/%d')
df['end'] = pd.to_datetime(df['end'], format='%Y/%m/%d')

dfgroup = df

df

,plan_id,employee,start,end,hours_day,employee_day
0,607,rasmus.paltschik,2023-04-19,2023-05-31,4.500000,NaN
1,541,juha.patrikainen,2023-01-01,2023-06-30,6.000000,NaN
2,376,1,2023-01-01,2023-12-31,4.615278,NaN
3,599,rasmus.paltschik,2023-04-20,2023-04-20,1.000000,NaN
4,533,jukka.haavisto,2023-04-01,2023-06-30,0.750000,NaN
...,...,...,...,...,...,...
67,586,arto.kovalainen,2023-05-08,2023-05-10,1.000000,NaN
68,553,tuomas.kara,2023-04-19,2023-04-20,2.400000,NaN
69,549,tapio.tuomisto,2023-03-01,2023-04-30,0.500000,NaN
70,578,ilkka.vaisanen,2023-04-17,2023-06-02,1.000000,NaN


In [290]:




employee_data = df.copy()

""" 

# Create a new data frame with every day of the project
date_range = pd.date_range(start=employee_data['start'].min(), end=employee_data['end'].max(), freq='D')
project_dates = pd.DataFrame({'Project Dates': date_range})

# Merge the new data frame with the employee data
merged_data = pd.merge(project_dates, employee_data, how='left', left_on='Project Dates', right_on='start')

# Fill in missing employees
merged_data['employee'].fillna(method='ffill', inplace=True)

# Create a new column with plan_id, employee and date for each day of the project for each employee
planned_id = merged_data['plan_id'].apply(str)
merged_data['Employee Project Dates'] = planned_id + ': ' + merged_data['employee'] + ': ' + merged_data['Project Dates'].dt.strftime('%Y-%m-%d')

# Pivot the data to create a new column for each employee
pivot_data = merged_data.pivot(index='Employee Project Dates', columns='employee', values='hours_day')

# Reset the index and rename the columns
pivot_data = pivot_data.reset_index().rename_axis(None, axis=1)

# Display the updated data frame
# print(pivot_data)

# Write the updated data frame to a new CSV file
merged_data.to_csv('updated_employee_data.csv', index=False)
 """

" \n\n# Create a new data frame with every day of the project\ndate_range = pd.date_range(start=employee_data['start'].min(), end=employee_data['end'].max(), freq='D')\nproject_dates = pd.DataFrame({'Project Dates': date_range})\n\n# Merge the new data frame with the employee data\nmerged_data = pd.merge(project_dates, employee_data, how='left', left_on='Project Dates', right_on='start')\n\n# Fill in missing employees\nmerged_data['employee'].fillna(method='ffill', inplace=True)\n\n# Create a new column with plan_id, employee and date for each day of the project for each employee\nplanned_id = merged_data['plan_id'].apply(str)\nmerged_data['Employee Project Dates'] = planned_id + ': ' + merged_data['employee'] + ': ' + merged_data['Project Dates'].dt.strftime('%Y-%m-%d')\n\n# Pivot the data to create a new column for each employee\npivot_data = merged_data.pivot(index='Employee Project Dates', columns='employee', values='hours_day')\n\n# Reset the index and rename the columns\npivot_da

In [291]:


df['date_range'] = df.apply(lambda row: pd.date_range(start=row['start'], end=row['end'], freq='D'), axis=1)

df_exploded = df.explode('date_range')

df_summed = df_exploded.groupby(['date_range', 'employee'])['hours_day'].sum().reset_index()
# df_summed
plans = df_exploded.groupby(['date_range', 'employee'])['plan_id'].agg(lambda x: list(x)).reset_index()
# Merge the new data frame with the employee data
merged_data = pd.merge(df_summed, plans, how='left')
# Read in the date table
date_table = pd.read_csv('Datetable.csv', parse_dates=['Date'])
# Convert the date column to a datetime object
date_table['Date'] = pd.to_datetime(date_table['Date'])

date_table.set_index('Date', inplace=True)


df['date'] = df.apply(lambda row: pd.date_range(start=row['start'], end=row['end'], freq='D'), axis=1)

df_exploded = df.explode('date')

df_hours_sum = df_exploded.groupby(['date', 'employee'])['hours_day'].sum().reset_index()
df_plan_agg = df_exploded.groupby(['date', 'employee'])['plan_id'].agg(lambda x: list(x)).reset_index()

df_merged = pd.merge(df_hours_sum, df_plan_agg, on=['date', 'employee'])
# Read in the date table and employee data table
date_df = pd.read_csv('Datetable.csv')

begin = '1/1/2023'
end = '31/12/2023'

all_dates = pd.date_range(start=begin, end=end, freq='D')
all_names = df_merged['employee'].unique()
name_date_df = pd.MultiIndex.from_product([all_names, all_dates], names=['employee', 'date']).to_frame(index=False)

df_all_days = pd.merge(name_date_df, df_merged, on=['employee', 'date'], how='left').fillna({'hours_day': 0})

df_all_days['date'] = df_all_days['date'].dt.floor('D')
df_all_days[df_all_days['employee'] == 'tuomas.kara']

""" # Create a new dataframe to hold the merged data
merged_df = pd.DataFrame()

# Loop through each employee in the employee data table
for index, row in employee_data.iterrows():
    # Create a new dataframe to hold the merged data for this employee
    employee_merged_df = pd.DataFrame()
    
    # Get the date range for this employee
    date_values = pd.date_range(start=row['start'], end=row['end'])
    
    # Create a new dataframe with the date range
    dates_df = pd.DataFrame({'Date': date_values})
    
    
    
    # Merge the date dataframe with the employee allocations for this employee
    employee_allocations_df = pd.DataFrame(row['Date'], columns=['Allocation'])
    # dates_with_allocations_df = pd.merge(dates_df, employee_allocations_df, how='left', left_on='Date', right_index=True)
    
    # Fill any missing allocations with 0
    # dates_with_allocations_df.fillna(0, inplace=True)
    
    # Add the employee name to the dataframe
    # dates_with_allocations_df['Employee'] = row['Employee Name']
    
    # Append the merged data for this employee to the overall merged dataframe
    # merged_df = pd.concat([merged_df, dates_with_allocations_df])



# Write the merged data to a new CSV file
# merged_df.to_csv('merged_data.csv', index=False)
 """
# row.info()
# dates_df.info()
merged_data = df_all_days.copy()

merged_data['allocation'] = 'Partly allocated'

merged_data.loc[merged_data['hours_day'] >= 7.5, 'allocation'] = 'Fully allocated'
merged_data.loc[merged_data['hours_day'] <= 0.1, 'allocation'] = 'Not allocated'

""" merged_data['coefficient'] = 0.5

merged_data.loc[merged_data['hours_day'] >= 7.5, 'coefficient'] = 1.0
merged_data.loc[merged_data['hours_day'] <= 0.1, 'coefficient'] = 0.0 """
# merged_data.loc[merged_data['hours_day'] < 7.5 and merged_data['hours_day'] > 0.1 , 'allocation'] = 'Partly allocated'
# merged_data['allocation'] = ''



# rowvalue
# plans
# df_summed
# merged_data

# Write the updated data frame to a new CSV file
merged_data.to_csv('updated_employee_data.csv', index=False)


C:\Users\JariKaijansinkko\AppData\Local\Temp\ipykernel_5088\1902589200.py:32: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  all_dates = pd.date_range(start=begin, end=end, freq='D')


In [292]:
# merged_data.groupby('employee').sum()

# pivot_data

# date_range

In [293]:
res = requests.get(f'{base_url}/rest/tempo-teams/2/team', headers=header)
teams = res.json()

In [294]:
team_members = requests.get(f'{base_url}/rest/tempo-teams/2/team/42/member', headers=header).json()
for member in team_members:
    print(member['member']['name'])

jani.lundan
jarkko.sillanpaa
juha.patrikainen
juho.lehtonen
jukka.haavisto
jyri.tienhaara
kai.jokiniemi
matias.ijas
mika.tavi
tapio.tuomisto
teemu.partanen
tuomas.kara
vilma.pohjonen
